In [1]:
from flask import Flask, render_template, request, jsonify
import requests
import torch

In [2]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from conversation import conv_templates, SeparatorStyle

from PIL import Image
import requests
import torch
import copy
import json
import time

/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
pretrained = "../../../models/multimodal/llama3-llava-next-8b"
model_name = "llava_llama3"
device = "cuda"
device_map = "auto"
tokenizer, model, image_processor, max_length = load_pretrained_model(pretrained, None, model_name, device_map=device_map) # Add any other thing you want to pass in llava_model_args

model.eval()
model.tie_weights()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

torch usage:	 16.84
total usage:	 17.29
total mem:	 25.39
torch reserved:	 16.86


In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_init_text')
def get_init_text():
    returned_content = [
        {'role': 'System', 'content': 'Hello, I am the system.'},
    ]
    return jsonify({'status': 'success', 'returned_content': returned_content})

@app.route('/send_user_msg', methods=['POST'])
def send_user_msg():
    data = request.get_json()
    user = data['user']
    content = data['content']

    url = f"http://127.0.0.1:10000/infer"
    data_to_send = {}
    data_to_send['content'] = content
    data_to_send['use_image'] = False
    response = requests.post(url, json=data_to_send)    
    ai_reply = response.json()['returned_content']
    
    returned_content = [
        {'role': 'User', 'content': content},
        {'role': 'AI', 'content': "\n".join(ai_reply)}
    ]
    print(f"Received text from {user}: {content}")
    return jsonify({'status': 'success', 'returned_content': returned_content})

if __name__ == '__main__':
    app.run(port=14000)